In [1]:
!pip install pysentiment2
!pip install pickle5
!pip install transformers

     |████████████████████████████████| 1.9 MB 12.0 MB/s 
     |████████████████████████████████| 256 kB 12.7 MB/s 
     |████████████████████████████████| 4.2 MB 12.1 MB/s 
     |████████████████████████████████| 6.6 MB 45.7 MB/s 
     |████████████████████████████████| 84 kB 3.1 MB/s 
     |████████████████████████████████| 596 kB 57.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk import tokenize
import re
import torch
from tqdm import tqdm
import random
import pysentiment2 as ps
from google.colab import files

#import 10KQ

unpickled_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DATA/IDX_FILE_TEXT_DAT.csv')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
unpickled_df = unpickled_df.reset_index(drop = True)

In [7]:
unpickled_df["negative"] = np.nan
unpickled_df["neutral"] = np.nan
unpickled_df["positive"] = np.nan

unpickled_df['Positive'] = np.nan
unpickled_df["Negative"] = np.nan
unpickled_df['Polarity'] = np.nan
unpickled_df['Subjectivity'] = np.nan

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone", model_max_len=512, truncation=True)

model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model.to(device)

nlp = pipeline('sentiment-analysis',model=model, tokenizer=tokenizer, device = 0)

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [10]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon May 23 07:45:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    36W / 250W |   1517MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
random.seed(3457)

dictio_finbert = {}
dictio_LM = {}

lm = ps.LM()

for i in tqdm(range(len(unpickled_df))):
  
  try:
      y = re.sub(r"[^a-zA-Z0-9 ]", " ", unpickled_df['text'][i])
      y = y.lower()
      y = re.sub(' {2,}', ' ', y)
      
      tokens = lm.tokenize(y)
      a = lm.get_score(tokens)   

      for r in a:
        unpickled_df.loc[i,r] = a[r]
      
      x = tokenize.sent_tokenize(unpickled_df['text'][i])
      
      x = [re.sub(r"[^a-zA-Z0-9 ]", " ", i) for i in x]

      x = [i.lower() for i in x]

      x = [re.sub(' {2,}', ' ', i) for i in x]
      
      x_new = []

      for o in range(len(x)):
          if len(x[o]) <= 1:
              continue
          elif len(x[o].split()) > 60:
              continue     
          else:
              x_new.append(x[o])     

      z = []
      k = ''
      for p in x_new:
          if (len(tokenizer(k)[0]) + len(tokenizer(p)[0]))<150:
              p = ' ' + p
              k += p
          else:
              k = k[1:]
              if k != '':
                  if k[0] == ' ':
                      k = k[2:]
                      z.append(re.sub(' {2,}', ' ', k))
                      k = ''
                  else:
                      z.append(re.sub(' {2,}', ' ', k))   
                      k = ''
              else:
                  continue
          
      z = random.sample(z, int((len(z)/100)*100))
      
      label_list = []
      score_list = []
      
      for h in z:
          result = nlp(h)
          label = result[0]['label']
          score = result[0]['score']
          label_list.append(label)
          score_list.append(score)
          
      dictio_finbert[i] = [label_list,score_list]
              
      values, counts = np.unique(label_list, return_counts=True)

      ziped = zip(values, counts)

      ziped_list = list(ziped)

      print(list(ziped))


      if len(ziped_list) > 0:
        for c in ziped_list:

          unpickled_df.loc[i,c[0].lower()] = c[1]

      unpickled_df.loc[i, 'text'] = np.nan
    
  except:
      continue


import pickle

pickle_out = open('/content/drive/MyDrive/Colab Notebooks/DATA/10kq_sentiment_new_yihang.pkl', "wb")
pickle.dump(unpickled_df,pickle_out)
pickle_out.close()


  0%|          | 0/12451 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:998: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
  0%|          | 1/12451 [00:02<7:51:35,  2.27s/it]

[]


  0%|          | 2/12451 [00:08<15:28:20,  4.47s/it]

[]


  0%|          | 3/12451 [00:09<10:34:02,  3.06s/it]

...

[]


  0%|          | 15/12451 [00:34<4:46:09,  1.38s/it]

[]


  0%|          | 16/12451 [00:34<3:57:31,  1.15s/it]

[]


In [ ]:


import pickle

pickle_out = open('/content/drive/MyDrive/Colab Notebooks/DATA/output_6_11000_13068_10kq_sentiment.pkl', "wb")
pickle.dump(output_6,pickle_out)
pickle_out.close()


In [ ]:
from google.colab import files

unpickled_df.to_pickle('output.pkl') 
files.download('output.pkl')